In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [2]:
#Define Model
class NN(nn.Module):
    def __init__(self,num_classes,input_size):
        super(NN,self).__init__()
        self.fp1=nn.Linear(input_size,70)
        self.fp2=nn.Linear(70,30)
        self.fp3=nn.Linear(30,num_classes)
    def forward(self,x):
        x=F.relu(self.fp1(x))
        x=F.relu(self.fp2(x))
        x=self.fp3(x)
        return x

In [3]:
#Hyperparameters
batch_size=128
num_classes=10
input_size=28*28
learning_rate=0.01
num_epoch=10

In [4]:
#Load train Data
train_data=datasets.FashionMNIST(root='datasets/',train=True,download=True,transform=transforms.ToTensor())
train_loader=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)

In [5]:
#Load test Data
test_data=datasets.FashionMNIST(root='datasets/',train=False,download=True,transform=transforms.ToTensor())
test_loader=DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)

In [6]:
#Selcting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
#Initalize model
model=NN(num_classes=num_classes,input_size=input_size).to(device)

In [8]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adagrad(model.parameters(),lr=learning_rate)

In [16]:
for epoch in range(num_epoch):
    for data_idx, (data,target) in enumerate(train_loader):
        #load data to cuda if availble
        data=data.to(device=device)
        target=target.to(device=device)
        #Reshape the data 
        data=data.reshape(data.shape[0],-1)
        #forward pass
        score=model(data)
        #loss
        loss=criterion(score,target)
        #Update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch completed:{epoch+1}')
    print(f'Training accuracy:{accuracy(train_loader,model)}')
print(f'Final Test accuracy:{accuracy(test_loader,model)}')

Epoch completed:1
Training accuracy:1.0
Epoch completed:2
Training accuracy:1.0
Epoch completed:3
Training accuracy:1.0
Epoch completed:4
Training accuracy:1.0
Epoch completed:5
Training accuracy:1.0
Epoch completed:6
Training accuracy:1.0
Epoch completed:7
Training accuracy:1.0
Epoch completed:8
Training accuracy:1.0
Epoch completed:9
Training accuracy:1.0
Epoch completed:10
Training accuracy:1.0
Final Test accuracy:1.0


In [15]:
def accuracy(data_loader,model):
    num_correct=0
    num_instance=0
    model.eval()
    with torch.no_grad():
        for data,target in data_loader:
            data=data.to(device=device)
            target=target.to(device=device)
            data=data.reshape(data.shape[0],-1)
            score=model(data)
            _,pred=score.max(1)
            num_correct+=(target==pred).sum()
            num_instance+=pred.size(0)
    model.train()
    return num_correct/num_correct